In [2]:
# A01650806 - Angel Cueto González
# A01651073 - Martín Figueroa Padilla

import numpy as np
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
%matplotlib inline
print("Using TensorFlow Version %s" %tf.__version__)

C:\Users\vilo_\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow Version 1.10.0


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
validate = pd.read_csv('gender_submission.csv')

In [3]:
train = train.drop(['Name' ,'Ticket' , 'Fare', 'Cabin', 'Embarked'],axis=1)
test = test.drop(['Name', 'Ticket' , 'Fare', 'Cabin', 'Embarked'],axis=1)

train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch
0,1,0,3,male,22.0,1,0
1,2,1,1,female,38.0,1,0
2,3,1,3,female,26.0,0,0
3,4,1,1,female,35.0,1,0
4,5,0,3,male,35.0,0,0
5,6,0,3,male,NaN,0,0
6,7,0,1,male,54.0,0,0
7,8,0,3,male,2.0,3,1
8,9,1,3,female,27.0,0,2
9,10,1,2,female,14.0,1,0


In [4]:
train['Sex_binary']=train['Sex'].map({'male':1,'female':0})
test['Sex_binary']=test['Sex'].map({'male':1,'female':0})

In [5]:
train['Age'] = train['Age'].fillna(0)
test['Age'] = test['Age'].fillna(0)

averageAgeTrain= sum(train['Age']) / len(train['Age']) 
averageAgeTest= sum( test['Age']) / len(test['Age']) 

for i in range(0, len(train['Age'])):
    if(train['Age'][i] == 0):
        train['Age'][i] = averageAgeTrain
        
for i in range(0, len(test['Age'])):
    if(test['Age'][i] == 0):
        test['Age'][i] = averageAgeTest 

/Users/AngelCueto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/AngelCueto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [6]:
features = ['Pclass','Age','Sex_binary']
target = 'Survived'

In [7]:
X_train = np.array(train[features])
X_train

y_train = np.array(train[target])
y_train = y_train.reshape(-1,1)
y_train



array([[0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
    

In [8]:
X_test = np.array(test[features])
X_test

array([[ 3.        , 34.5       ,  1.        ],
       [ 3.        , 47.        ,  0.        ],
       [ 2.        , 62.        ,  1.        ],
       ...,
       [ 3.        , 38.5       ,  1.        ],
       [ 3.        , 24.04425837,  1.        ],
       [ 3.        , 24.04425837,  1.        ]])

In [9]:
# Definición de la estructura de la red:
nInputRows = X_train.shape[0] # cuantos renglones
nInputsColumns = X_train.shape[1] #cuantas columnas

# Capa
#nHiddenNeurons = 4 # número de neuronas
nOutputs = 1 # número de salidas, binario


In [16]:
class NeuronalNet(object):
    def __init__(self, sess, X, Y, n_hidden=20, learning_rate=1e-2, layers=5):
        self.sess = sess #Sesión para correr TensorFlow
        self.X = X # Entradas
        self.Y = Y.reshape(-1,1) #Salidas
        self.n_inputs = X.shape[0] # Renglones
        self.n_input_dim = X.shape[1] #Columnas
        self.n_output = 1 # Como es binario, sólo UNA salida
        self.learning_rate = learning_rate 
        self.layers = layers
        '''
        La tasa de aprendizaje es un hiper-parámetro que controla 
        cuánto estamos ajustando los pesos de nuestra red con respecto 
        al gradiente de pérdida. Cuanto más bajo sea el valor, 
        más lento viajamos a lo largo de la pendiente descendente. 
        Si bien esto podría ser una buena idea (usar una tasa de aprendizaje baja) 
        para asegurarnos de no perder ningún mínimo local, 
        también podría significar que nos llevará mucho tiempo converger, 
        especialmente si nos quedamos atascados. 
        '''
        self.n_hidden = n_hidden # número de neuronas 
        
        # Create NeuronalNet
        self.X_input, self.y, self.logits, self.cost = self.createNeuralNet()
        
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate) #optimizador

        self.train_op = self.optimizer.minimize(self.cost)#operación de entreanmiento


    #Metodo de construcción de la red:
    def createNeuralNet(self):
        # Investiga qué es el xavier_initializer???? Reporta en el código:
        initializer = tf.contrib.layers.xavier_initializer()

        # Agrega el placeholder para X_input, entradas
        X_input = tf.placeholder(shape=[None, self.n_input_dim], dtype=tf.float32, name='input')
        # Agrega el placeholder para y, salidas
        y = tf.placeholder(shape=[None, self.n_output], dtype=tf.float32, name='output')

        # Agrega una capa fully_connected de X_input * n_hidden, con una activación (activation_fn) tf.nn.elu, inicializa los pesos weights_initializer
        #Reporta qué es una función de activación tf.nn.elu
        _hiddens = []
        for i in range(0, self.layers):
            if (i == 0): hidden_input = X_input
            else: hidden_input = _hiddens[len(_hiddens) - 1]
            hidden_temp = tf.contrib.layers.fully_connected(hidden_input, self.n_hidden, activation_fn=tf.nn.elu, weights_initializer=initializer)
            _hiddens.append(hidden_temp)
        # Agrega una capa fully_connected de hidden1 * n_output ,
        #con una activación (activation_fn) tf.nn.sigmoid, inicializa los pesos weights_initializer
        logits = tf.contrib.layers.fully_connected(_hiddens[len(_hiddens) - 1], self.n_output, activation_fn=tf.nn.sigmoid, weights_initializer=initializer)
        #!!!!!!!!!!!!! calcula el error con tf.nn.sigmoid_cross_entropy_with_logits
        loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y, name='loss')
        #calcula el costo con tf.reduce_mean
        cost = tf.reduce_mean(loss)
        
        #Regresa X_input, y, logits, costo
        return X_input, y, logits, cost
    
    #funcion de entrenamiento 
    def train(self):
    	#ejecuta el entrenamiento debes correr train_op, cost y en el diccionario manda X_input, X , y, Y
        _, cost = self.sess.run([self.train_op,self.cost], feed_dict={ self.X_input: self.X, self.y: self.Y })        
        return cost
    
	#funcion de predicción
    #NO modificar, dudas con el código?
    def predict(self, X_test):
        pred = self.sess.run([self.logits], feed_dict={ self.X_input: X_test})[0]        
        return pred
    
    #función de graficación, NO modificar
    #Fuente: http://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html#sphx-glr-auto-examples-neural-networks-plot-mlp-alpha-py
    def plot_decision_boundary(self):
        x_min, x_max = self.X[:, 0].min()-0.1, self.X[:, 0].max()+0.1
        y_min, y_max = self.X[:, 1].min()-0.1, self.X[:, 1].max()+0.1
        spacing = min(x_max - x_min, y_max - y_min) / 100
        XX, YY = np.meshgrid(np.arange(x_min, x_max, spacing),
                       np.arange(y_min, y_max, spacing))
        data = np.hstack((XX.ravel().reshape(-1,1), 
                          YY.ravel().reshape(-1,1)))
        db_prob = self.predict(data)
        clf = np.where(db_prob<0.5,0,1)
        Z = clf.reshape(XX.shape)
        plt.figure(figsize=(10,8))
        plt.contourf(XX, YY, Z, cmap=plt.cm.Spectral, alpha=0.8)
        plt.scatter(X[:,0], X[:,1], c=Y, 
                    cmap=plt.cm.Spectral)
        plt.show()

In [17]:
#NO MODIFICAR
# función para obtener el costo del modelo al ejecutarlo n número de iteraciones
def getTrainCost(model, n_iters=1000):
    model.sess.run(tf.global_variables_initializer())
    cost = []
    for i in range(n_iters):
        cost.append(model.train())
    return cost

In [25]:
#NO MODIFICAR
n_iters = 1000
print("%d Neuronas" % 1)
tf.reset_default_graph() #BORRA EL GRAFO ANTERIOR EN LA SESIÓN
sess = tf.Session() #SESIÓN DE TENSORFLOW
net = NeuronalNet(sess, X_train, y_train, n_hidden=8)

cost = getTrainCost(net, n_iters) #COSTO DEL MODELO
pred_prob = net.predict(X_test) #PROBABILIDADES DE LA PREDICCIÓN

y_hat = np.where(pred_prob<0.5,0,1) #YHAT
print(y_hat)

#precision = np.sum(Y_test.reshape(-1,1)==y_hat) / len(Y_test) #CALCULAR LA PRECISIÓN
#print("Precisión: %.2f" % precision)
#net.plot_decision_boundary() #imprimir los datos

1 Neuronas
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]

In [27]:
validate['final'] = y_hat
validate





,PassengerId,Survived,final
0,892,0,0
1,893,1,0
2,894,0,0
3,895,0,0
4,896,1,0
5,897,0,0
6,898,1,0
7,899,0,0
8,900,1,0
9,901,0,0


In [28]:
toKaggle = pd.DataFrame({'PassengerId':validate['PassengerId'],
                         'Survived':validate['final']})

toKaggle.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [30]:
from datetime import datetime
archivo = 'TitanicTensonFlowFinal_'+ str(datetime.now().strftime('%Y-%m-%d'))+'.csv'

toKaggle.to_csv(archivo,index=False)

print('Creado: ' + archivo)

Creado: TitanicTensonFlowFinal_2018-10-29.csv
